# Main Project File
This is the main file for the project. It contains the core analysis and model development workflow.


In [ ]:
import pandas as pd
import numpy as np
from modelhelper import ModelHelper
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [ ]:
#Import the data and put it into a dataframe
data = pd.read_csv('data/data.csv')

#Initialze new modelhelper object
model_helper = ModelHelper()

#Identify Target Column
target_column = 'target'

In [ ]:
#Clean the data
cleaned_data = model_helper.clean_data(data, target_column)